In [1]:
pip install torch transformers datasets gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer, DataCollatorForLanguageModeling, GPT2Tokenizer
import gradio as gr

# Create a list of 30 emoji math problems with their solutions.
# Format: "Q: [emoji math equation]\nA: [solution]"
data = [
    "Q: 🍎 + 🍎 + 🍎 = 12\nA: 4",
    "Q: 🎲 + 🎲 = 12\nA: 6",
    "Q: 🚗 + 🚗 + 🚗 + 🚗 = 20\nA: 5",
    "Q: 🍌 + 🍌 + 🍌 + 🍌 + 🍌 = 15\nA: 3",
    "Q: 🍓 + 🍓 = 8\nA: 4",
    "Q: 🍕 + 🍕 + 🍕 = 18\nA: 6",
    "Q: 🍩 + 🍩 + 🍩 + 🍩 = 20\nA: 5",
    "Q: 🌟 + 🌟 + 🌟 = 9\nA: 3",
    "Q: 🎈 + 🎈 = 14\nA: 7",
    "Q: 🎂 + 🎂 + 🎂 = 15\nA: 5",
    "Q: 🍪 + 🍪 + 🍪 + 🍪 = 16\nA: 4",
    "Q: 🍭 + 🍭 + 🍭 = 15\nA: 5",
    "Q: 🧁 + 🧁 = 10\nA: 5",
    "Q: 🥑 + 🥑 + 🥑 = 12\nA: 4",
    "Q: 🍇 + 🍇 = 10\nA: 5",
    "Q: 🍒 + 🍒 + 🍒 = 15\nA: 5",
    "Q: 🍍 + 🍍 = 14\nA: 7",
    "Q: 🍉 + 🍉 + 🍉 + 🍉 = 20\nA: 5",
    "Q: 🥭 + 🥭 = 16\nA: 8",
    "Q: 🍈 + 🍈 + 🍈 = 9\nA: 3",
    "Q: 🍑 + 🍑 + 🍑 + 🍑 = 20\nA: 5",
    "Q: 🍏 + 🍏 = 10\nA: 5",
    "Q: 🍋 + 🍋 + 🍋 = 12\nA: 4",
    "Q: 🍊 + 🍊 = 10\nA: 5",
    "Q: 🥝 + 🥝 + 🥝 = 15\nA: 5",
    "Q: 🍐 + 🍐 = 8\nA: 4",
    "Q: 🍆 + 🍆 + 🍆 + 🍆 = 16\nA: 4",
    "Q: 🥕 + 🥕 = 10\nA: 5",
    "Q: 🌽 + 🌽 + 🌽 = 9\nA: 3",
    "Q: 🥔 + 🥔 + 🥔 + 🥔 = 20\nA: 5"
]

# For training with Hugging Face's datasets, we create a dictionary.
from datasets import Dataset
dataset = Dataset.from_dict({"text": data})

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(example):
    return tokenizer(example["text"], truncation=True, max_length=128, padding="max_length")

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.config.pad_token_id = tokenizer.eos_token_id

# Create a data collator for language modeling that will handle padding dynamically
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./emoji_math_model",
    overwrite_output_dir=True,
    num_train_epochs=8,
    per_device_train_batch_size=4,
    save_steps=50,
    save_total_limit=2,
    logging_steps=10,
    learning_rate=1e-5
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)

# Start training
trainer.train()

import logging
logging.getLogger("transformers").setLevel(logging.ERROR)  # Suppress transformer warnings

import re

def generate_single_answer(prompt, max_new_tokens=10):
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs["input_ids"].to(model.device)
    attention_mask = inputs["attention_mask"].to(model.device)

    output = model.generate(
        input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        repetition_penalty=2.0
    )

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)

    if prompt in generated_text:
        generated_text = generated_text.split(prompt, 1)[1]

    match = re.search(r'\b(\d+)\b', generated_text)
    if match:
        answer = match.group(1)
    else:
        answer = generated_text.strip()

    return answer

# Gradio UI
def gradio_interface(prompt):
    return generate_single_answer(prompt)

iface = gr.Interface(fn=gradio_interface, inputs="text", outputs="text", title="Emoji Math Solver", description="Enter an emoji math problem to get the answer.")
iface.launch()


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: reh1t (reh1t-fast-nuces) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,3.242900
20,2.277300
30,1.814200
40,1.520600
50,1.349400
60,1.162600


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f42dfbcbecc240e616.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
